It looks like the update rate of the IMU is non deterministic and lower than the rate the phone samples it at, i.e. the phone receives a non-deterministic number of sequential identical measurements from the IMU when polling at approx 200 Hz.

Datasheets of smartphone level IMUs suggest that IMU filters data before sending it to the phone, and that the cutoff frequency of this filtering is configurable and changes with update frequency.

Based on the IMU update rate of about 20 ms, this cutoff frequency is already close to 40 Hz, so filtering is probably unnecessary. Its hard to be sure about actual values because this is configured by the phone manufacturer.

In [1]:
'''Join acc and gyro data then export to csv'''
def join_export(acc_frame, gyro_frame, export_name):
    # Extract only the gyroscope data from the gyro dataframe
    # This assumes the time components of each are aligned
    join_frame = acc_frame.join(gyro_frame[['GYROSCOPE X (rad/s)', 'GYROSCOPE Y (rad/s)', 'GYROSCOPE Z (rad/s)']])
    path = 'imu_data/' + export_name
    join_frame.to_csv(path, index=False)

In [ ]:
# Run on everything

terrains = ['Concrete', 'Carpet', 'Linoleum']
locations = ['left', 'right', 'Frame9250']

for terrain in terrains:
    for location in locations:
        if (location == 'Frame9250'):
            filename = 'FrameMiddle' + terrain + '.csv'
        else:
            filename = 'Wheel' + location.capitalize() + terrain + '.csv'
        gyro = raw_datasets[location + 'Gyro' + terrain]
        acc = raw_datasets[location + 'Acc' + terrain]
        join_export(acc, gyro, filename)

In [ ]:
# Attempting to calculate the angular velocity of the wheelchair using the rotational velocity of each wheel

wheel_radius = 0.60 #m
wheel_sep = 0.502 #m

raw_datasets['CalculatedLinoleum'] = pd.DataFrame()

raw_datasets['calculatedLinoleum']['YYYY-MO-DD HH-MI-SS_SSS'] = raw_datasets['leftGyroLinoleum']['YYYY-MO-DD HH-MI-SS_SSS']
raw_datasets['calculatedLinoleum']['GYROSCOPE Z (rad/s)'] = raw_datasets['leftGyroLinoleum']['GYROSCOPE Z (rad/s)'] - raw_datasets['rightGyroLinoleum']['GYROSCOPE Z (rad/s)'] 
raw_datasets['calculatedLinoleum']['GYROSCOPE Z (rad/s)'] = raw_datasets['calculatedLinoleum']['GYROSCOPE Z (rad/s)'].apply(lambda x: x*(wheel_radius/wheel_sep))

label_compare('calculatedLinoleum', 'Frame9250GyroLinoleum', 'GYROSCOPE Z (rad/s)')

In [ ]:
# Convert time received to time since start
for label in ('Concrete', 'Carpet', 'Linoleum'):
    for mount in ('WheelLeft', 'WheelRight'):
        start = Decimal(raw_datasets[mount+label]['Time received in s'][0])
        raw_datasets[mount+label]['Time received in s'] = raw_datasets[mount+label]['Time received in s'].apply(lambda x: float((Decimal(x) - start) * 1000))
        raw_datasets[mount+label].rename(columns = {'Time received in s': 'Time since start in ms '}, inplace=True)

In [ ]:
# Standardize order of columns
for label in raw_datasets.keys():
    raw_datasets[label] = raw_datasets[label][get_columns(label)]

In [ ]:
# Export everything to CSV
for label, dataset in raw_datasets.items():
    dataset.to_csv('imu_data/'+label+'.csv', index=False)

In [ ]:
'''Compare time domain data from phones mounted in different locations'''
def phone_compare(dirn, surface):
    plt.clf()
    fig = plt.figure(figsize=(30, 10))
    
    # Use epoch times to align data
    for mount in ('Left', 'Right', 'Middle'):
        plt.plot(raw_datasets['Phone' + mount + surface]['Epoch Time'],
                 raw_datasets['Phone' + mount + surface][dirn], label=mount)
    
    plt.xlabel('Epoch Time (s)')
    plt.ylabel(dirn + ' on ' + surface)
    plt.legend()
    plt.show()

In [ ]:
# Convert string format to epoch time (newer data only)
    if any(s in dataset_label for s in ('Phone', 'Asphalt', 'Grass', 'Gravel', 'Sidewalk')):
        dataset['Epoch Time'] = dataset['Epoch Time'].apply(datetime.strptime, 
                                                            args=("%Y-%m-%d %H:%M:%S:%f",))
        dataset['Epoch Time'] = dataset['Epoch Time'].apply(datetime.timestamp)

In [ ]:
# Need to convert everything to a single pandas dataset
def to_single_dataframe(datasets, feat_names):
    # Create the columns of the array
    
    columns = ['Label']
    for j in range(N_DATA_COL):
        for name in feat_names:
            columns.append(dataset_columns[j] + ' ' + name)
    dataframe = pd.DataFrame(columns=columns, index=[])
    
    for i, dataset in enumerate(datasets):
        set_dataframe = pd.DataFrame(columns)
        
        for j, direction in enumerate(dataset[1:]):
            # Direction labelled columns
            direction_columns = ['Label']
            for name in feat_names:
                direction_columns.append(dataset_columns[j] + ' ' + name)
            
            # Data directions into a single dataframe
            direction.columns = direction_columns
            set_dataframe = set_dataframe.append(direction)
            
        # Combine datasets into a single dataframe
        dataframe.append(dataset)
    
    return dataframe

In [ ]:
'Add labels to a dataset'
def insert_labels(datasets, windowed=False):
    # Add to each dataframe of a dataset
    for i, (label, dataset) in enumerate(datasets.items()):
        # Either formatted in windows (transforms) or dict->dict (features)
        if (windowed):
            for window_df in dataset:
                labels = [get_terrain(label) for _ in range(len(window_df))]
                window_df.insert(0, 'Label', labels)
                window_df.set_index('Label')
        
        else:
            for dirn_label, dirn_df  in dataset.items():
                labels = [get_terrain(label) for _ in range(len(dirn_df))]
                dirn_df.insert(0, 'Label', labels)
                dirn_df.set_index('Label')

In [ ]:
'''Set indices of a dataframe depending on given label'''
def set_indices(dataframe):
    if 'Frequency' in dataframe.columns:
        return dataframe.set_index('Frequency')
    elif 'Epoch Time' in dataframe.columns:
        return dataframe.set_index('Epoch Time')
    else:
        raise('Unknown dataframe columns')

In [ ]:
'''Move frequency column to index'''
def set_freq_indices(datasets):
    for dataset in datasets.values():
        for window in dataset:
            window = window.set_index('Frequency')

In [ ]:
'''Get recording range for speed tests'''
def get_range(label):
    # Recording ranges for each sampled terrain
    ranges = {'AsphaltSlow_':   (2400, 4800), 'AsphaltSlow2_':   (3400, 6200),
              'AsphaltMedium_': (3000, 5600), 'AsphaltFast_':    (1600, 3600),
              'ConcreteFast_':  (1700, 5400), 'ConcreteMedium_': (2400, 4800),
              'ConcreteSlow_':  (4000, 6800),
              'AsphaltPowerRemoteMedium_': (2000, 10000),
              'AsphaltPowerRemoteFast_': (1400, 5600),
              'ConcretePowerRemoteFast_': (1600, 5600),
              'ConcretePowerRemoteFast2_': (1400, 5200),
              'ConcretePowerRemoteMedium_': (4000, 14000)}

    for range_label, (start, stop) in ranges.items():
        if range_label in label:
            return start, stop
    
    raise ValueError('Unknown label ' + label)

In [ ]:
for label, dataset in datasets.items():
    start, stop = get_range(label)
    print(label + ' {}'.format(np.sqrt(np.mean(np.power(dataset[start:stop, 5], 2)))))

In [ ]:
# Combine all data individually
# TODO: Combine each direction of a single dataset into a single dataframe so we can select across 
datasets_middle = {label: dataset for label, dataset in datasets_psd_log_combined.items() if 'Middle' in label}
datasets_left = {label: dataset for label, dataset in datasets_psd_log_combined.items() if 'Left' in label}
datasets_right = {label: dataset for label, dataset in datasets_psd_log_combined.items() if 'Right' in label}

middle_psd_logs_all = combine_datasets(datasets_middle)
left_psd_logs_all = combine_datasets(datasets_left)
right_psd_logs_all = combine_datasets(datasets_right)

# Export to csv
middle_psd_logs_all.to_csv('featured_data/Middle_PSDLogs.csv', index=False)
left_psd_logs_all.to_csv('featured_data/Left_PSDLogs.csv', index=False)
right_psd_logs_all.to_csv('featured_data/Right_PSDLogs.csv', index=False)

# Drop any row with a NaN
middle_psd_logs_all = middle_feats_all.dropna()
left_psd_logs_all = left_feats_all.dropna()
right_psd_logs_all = right_feats_all.dropna()